In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from sklearn.datasets import make_multilabel_classification
import numpy as np


In [4]:
!pip install nvidia-ml-py3
!pip install scikit-multilearn
!pip install evaluate
!pip install datasets
! pip install -U accelerate
! pip install -U transformers

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=92a41d9437dc82c65cf3c3a8e949e32b94e09e4c0a3a50d75399bd98c6592759
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 

In [5]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [6]:
print_gpu_utilization()

GPU memory occupied: 233 MB.


In [7]:

answers = pd.read_csv("drive/MyDrive/Levy/english_texts.csv")

In [8]:

answers

,Unnamed: 0,TEXT,TEXT_NL,cEXT,cNEU,cAGR,cCON,cOPN
0,0,"Well, right now I just woke up from a mid-day ...","Nou, op dit moment werd ik net wakker na een m...",0,1,1,0,1
1,1,"Well, here we go with the stream of consciousn...","Nou, hier gaan we met de stroom van bewustzijn...",0,0,1,0,0
2,2,An open keyboard and buttons to push. The thin...,Een open toetsenbord en knoppen om in te drukk...,0,1,0,1,1
3,3,I can't believe it! It's really happening! M...,Ik kan het niet geloven! Het gebeurt echt! Mij...,1,0,1,1,0
4,4,"Well, here I go with the good old stream of co...","Welnu, hier ga ik weer met de goede oude stroo...",1,0,1,0,1
...,...,...,...,...,...,...,...,...
2958,2958,I am motivated on a day to day basis by the ne...,Ik word dagelijks gemotiveerd door de noodzaak...,1,0,0,1,1
2959,2959,My son is the biggest part of my life and with...,Mijn zoon is het grootste deel van mijn leven ...,1,1,0,0,0
2960,2960,My kids and grandkids are what keeps me motiva...,Mijn kinderen en kleinkinderen houden me elke ...,1,0,1,1,0
2961,2961,My biggest drive is to earn money so I can ret...,Mijn grootste drijfveer is om geld te verdiene...,0,0,0,0,0


In [9]:
from skmultilearn.model_selection import iterative_train_test_split

x = answers['TEXT']
y = answers[['cEXT',	'cNEU',	'cAGR',	'cCON',	'cOPN']]


# Convert the labels DataFrame to a numpy array
y_array = y.to_numpy()

# Iterative stratification to split the dataset
x_array = x.to_numpy().reshape(-1, 1)
x_train_it, y_train_it, x_test_it, y_test_it = iterative_train_test_split(x_array, y_array, test_size = 0.2)

# Display the shapes of the train and test sets after iterative stratification
print(x_train_it.shape, x_test_it.shape, y_train_it.shape, y_test_it.shape)


(2370, 1) (593, 1) (2370, 5) (593, 5)


In [10]:
# Creating a list of dictionaries for the test set
test_data = [{'label': label, 'text': text} for label, text in zip(y_test_it, x_test_it)]
train_data = [{'label': label, 'text': text} for label, text in zip(y_train_it, x_train_it)]

# Creating the final data structure
multilabel_df = {
    'train': train_data,
    'test': test_data
}

In [11]:
multilabel_df['train'][0]

{'label': array([0, 1, 1, 0, 1]),
 'text': array(['Well, right now I just woke up from a mid-day nap. It\'s sort of weird, but ever since I moved to Texas, I have had problems concentrating on things. I remember starting my homework in  10th grade as soon as the clock struck 4 and not stopping until it was done. Of course it was easier, but I still did it. But when I moved here, the homework got a little more challenging and there was a lot more busy work, and so I decided not to spend hours doing it, and just getting by. But the thing was that I always paid attention in class and just plain out knew the stuff, and now that I look back, if I had really worked hard and stayed on track the last two years without getting  lazy, I would have been a genius, but hey, that\'s all good. It\'s too late to correct the past, but I don\'t really know how to stay focused n the future. The one thing I know is that when  people say that b/c they live on campus they can\'t concentrate, it\'s b. s. For

In [12]:
def label_distribution(data):
    # Initialize counts for each label
    label_counts = {i: {'1': 0, '0': 0} for i in range(5)}

    # Iterate over each entry and count the label occurrences
    for entry in data:
        labels = entry['label']
        for i, label in enumerate(labels):
            label_str = str(int(label))  # Convert label to string (either '1' or '0')
            label_counts[i][label_str] += 1

    return label_counts

# Calculating label distributions for train and test sets
train_label_distribution = label_distribution(multilabel_df['train'])
test_label_distribution = label_distribution(multilabel_df['test'])

train_label_distribution, test_label_distribution

({0: {'1': 1193, '0': 1177},
  1: {'1': 1186, '0': 1184},
  2: {'1': 1230, '0': 1140},
  3: {'1': 1214, '0': 1156},
  4: {'1': 1145, '0': 1225}},
 {0: {'1': 298, '0': 295},
  1: {'1': 296, '0': 297},
  2: {'1': 307, '0': 286},
  3: {'1': 304, '0': 289},
  4: {'1': 286, '0': 307}})

In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [40]:
import torch

def tokenize_and_chunk(texts, labels, tokenizer, chunk_size=510):
    input_id_chunks = []
    mask_chunks = []
    chunked_labels = []

    for text, label in zip(texts, labels):
        # Ensure text is a string
        text = str(text)

        # Tokenize without special tokens
        tokens = tokenizer.encode_plus(text, add_special_tokens=False,
                                       return_tensors='pt', truncation=False)

        # Split into chunks of `chunk_size`
        input_ids = tokens['input_ids'][0]
        attention_mask = tokens['attention_mask'][0]
        num_chunks = (len(input_ids) // chunk_size) + int(len(input_ids) % chunk_size != 0)

        for i in range(num_chunks):
            # Define the start and end of the chunk
            start = i * chunk_size
            end = start + chunk_size

            # Extract chunks for input IDs and attention mask
            input_ids_chunk = input_ids[start:end]
            attention_mask_chunk = attention_mask[start:end]

            # Add [CLS] and [SEP] tokens
            input_ids_chunk = torch.tensor([101] + input_ids_chunk.tolist() + [102])
            attention_mask_chunk = torch.tensor([1] + attention_mask_chunk.tolist() + [1])

            # Pad the sequences
            padding_length = 512 - len(input_ids_chunk)
            input_ids_chunk = torch.cat([input_ids_chunk, torch.zeros(padding_length, dtype=torch.long)])
            attention_mask_chunk = torch.cat([attention_mask_chunk, torch.zeros(padding_length, dtype=torch.long)])

            # Store the chunks
            input_id_chunks.append(input_ids_chunk)
            mask_chunks.append(attention_mask_chunk)
            chunked_labels.append(torch.tensor(label, dtype=torch.float))  # Convert label list to tensor of floats

    # Convert lists to tensors
    input_ids_tensor = torch.stack(input_id_chunks)
    attention_mask_tensor = torch.stack(mask_chunks)
    labels_tensor = torch.stack(chunked_labels)  # Stack label tensors

    return input_ids_tensor, attention_mask_tensor, labels_tensor


# Training data
train_texts = [item['text'] for item in multilabel_df['train']]
train_labels = [item['label'] for item in multilabel_df['train']]
input_ids_tensor_train, attention_mask_tensor_train, labels_tensor_train = tokenize_and_chunk(train_texts, train_labels, tokenizer)

# Preparing the training dictionary
input_dict_train = {
    'input_ids': input_ids_tensor_train.long(),
    'attention_mask': attention_mask_tensor_train.int(),
    'labels': labels_tensor_train
}

# Testing data
test_texts = [item['text'] for item in multilabel_df['test']]
test_labels = [item['label'] for item in multilabel_df['test']]
input_ids_tensor_test, attention_mask_tensor_test, labels_tensor_test = tokenize_and_chunk(test_texts, test_labels, tokenizer)

# Preparing the testing dictionary
input_dict_test = {
    'input_ids': input_ids_tensor_test.long(),
    'attention_mask': attention_mask_tensor_test.int(),
    'labels': labels_tensor_test
}


In [41]:
import pandas as pd
from datasets import Dataset

# Function to convert tensor to list of integers or floats
def tensor_to_list(tensor, dtype=int):
    return [tensor[i].tolist() for i in range(len(tensor))]

# Convert tensors to lists
input_ids_list_train = tensor_to_list(input_ids_tensor_train, dtype=int)
attention_mask_list_train = tensor_to_list(attention_mask_tensor_train, dtype=int)
labels_list_train = tensor_to_list(labels_tensor_train, dtype=float)

input_ids_list_test = tensor_to_list(input_ids_tensor_test, dtype=int)
attention_mask_list_test = tensor_to_list(attention_mask_tensor_test, dtype=int)
labels_list_test = tensor_to_list(labels_tensor_test, dtype=float)

# Create DataFrame
df_train = pd.DataFrame({
    'input_ids': input_ids_list_train,
    'attention_mask': attention_mask_list_train,
    'labels': labels_list_train
})

df_test = pd.DataFrame({
    'input_ids': input_ids_list_test,
    'attention_mask': attention_mask_list_test,
    'labels': labels_list_test
})

# Create Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)


In [42]:
train_dataset['input_ids'][0]

[101,
 48759,
 8346,
 6,
 235,
 122,
 38,
 95,
 13356,
 62,
 31,
 10,
 1084,
 12,
 1208,
 16159,
 4,
 85,
 46495,
 29,
 2345,
 9,
 7735,
 6,
 53,
 655,
 187,
 38,
 1410,
 7,
 1184,
 6,
 38,
 33,
 56,
 1272,
 28619,
 15,
 383,
 4,
 38,
 2145,
 1158,
 127,
 19122,
 11,
 1437,
 158,
 212,
 4978,
 25,
 1010,
 25,
 5,
 6700,
 2322,
 204,
 8,
 45,
 8197,
 454,
 24,
 21,
 626,
 4,
 1525,
 768,
 24,
 21,
 3013,
 6,
 53,
 38,
 202,
 222,
 24,
 4,
 125,
 77,
 38,
 1410,
 259,
 6,
 5,
 19122,
 300,
 10,
 410,
 55,
 4087,
 8,
 89,
 21,
 10,
 319,
 55,
 3610,
 173,
 6,
 8,
 98,
 38,
 1276,
 45,
 7,
 1930,
 722,
 608,
 24,
 6,
 8,
 95,
 562,
 30,
 4,
 125,
 5,
 631,
 21,
 14,
 38,
 460,
 1199,
 1503,
 11,
 1380,
 8,
 95,
 10798,
 66,
 1467,
 5,
 2682,
 6,
 8,
 122,
 14,
 38,
 356,
 124,
 6,
 114,
 38,
 56,
 269,
 1006,
 543,
 8,
 4711,
 15,
 1349,
 5,
 94,
 80,
 107,
 396,
 562,
 1437,
 22414,
 6,
 38,
 74,
 33,
 57,
 10,
 16333,
 6,
 53,
 17232,
 6,
 14,
 46495,
 29,
 70,
 205,
 4,
 85,
 46495,
 29

In [43]:
train_dataset['labels'][0]

[0.0, 1.0, 1.0, 0.0, 1.0]

In [44]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [46]:
labels = ['cEXT',	'cNEU',	'cAGR',	'cCON',	'cOPN']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']

In [47]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [48]:
train_dataset['labels'][0]

[0.0, 1.0, 1.0, 0.0, 1.0]

In [49]:
train_dataset['input_ids'][0]

[101,
 48759,
 8346,
 6,
 235,
 122,
 38,
 95,
 13356,
 62,
 31,
 10,
 1084,
 12,
 1208,
 16159,
 4,
 85,
 46495,
 29,
 2345,
 9,
 7735,
 6,
 53,
 655,
 187,
 38,
 1410,
 7,
 1184,
 6,
 38,
 33,
 56,
 1272,
 28619,
 15,
 383,
 4,
 38,
 2145,
 1158,
 127,
 19122,
 11,
 1437,
 158,
 212,
 4978,
 25,
 1010,
 25,
 5,
 6700,
 2322,
 204,
 8,
 45,
 8197,
 454,
 24,
 21,
 626,
 4,
 1525,
 768,
 24,
 21,
 3013,
 6,
 53,
 38,
 202,
 222,
 24,
 4,
 125,
 77,
 38,
 1410,
 259,
 6,
 5,
 19122,
 300,
 10,
 410,
 55,
 4087,
 8,
 89,
 21,
 10,
 319,
 55,
 3610,
 173,
 6,
 8,
 98,
 38,
 1276,
 45,
 7,
 1930,
 722,
 608,
 24,
 6,
 8,
 95,
 562,
 30,
 4,
 125,
 5,
 631,
 21,
 14,
 38,
 460,
 1199,
 1503,
 11,
 1380,
 8,
 95,
 10798,
 66,
 1467,
 5,
 2682,
 6,
 8,
 122,
 14,
 38,
 356,
 124,
 6,
 114,
 38,
 56,
 269,
 1006,
 543,
 8,
 4711,
 15,
 1349,
 5,
 94,
 80,
 107,
 396,
 562,
 1437,
 22414,
 6,
 38,
 74,
 33,
 57,
 10,
 16333,
 6,
 53,
 17232,
 6,
 14,
 46495,
 29,
 70,
 205,
 4,
 85,
 46495,
 29

In [50]:
[id2label[idx] for idx, value in enumerate(train_dataset['labels'][0]) if value == 1.0]


['cNEU', 'cAGR', 'cOPN']

In [51]:
output_dir = '/content/drive/MyDrive/Levy/part2/roberta'


In [52]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error()

In [53]:
print_gpu_utilization()

GPU memory occupied: 5725 MB.


In [54]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [55]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_steps = 20,
    logging_steps = 8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    disable_tqdm = False,
    load_best_model_at_end=True,
    push_to_hub=False,
    gradient_accumulation_steps=32,
    gradient_checkpointing=True,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
0,0.693500,0.691598,0.622071,0.508490,0.027289
2,0.656600,0.676237,0.626175,0.571581,0.060739
4,0.641300,0.681267,0.626275,0.572846,0.056338


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=180, training_loss=0.6652198142475552, metrics={'train_runtime': 554.778, 'train_samples_per_second': 42.08, 'train_steps_per_second': 0.324, 'total_flos': 6059084584660992.0, 'train_loss': 0.6652198142475552, 'epoch': 4.93})

In [56]:
print_gpu_utilization()

GPU memory occupied: 5857 MB.


In [57]:
trainer.evaluate()

{'eval_loss': 0.676237165927887,
 'eval_f1': 0.6261754277786342,
 'eval_roc_auc': 0.5715806373974086,
 'eval_accuracy': 0.06073943661971831,
 'eval_runtime': 5.1379,
 'eval_samples_per_second': 221.1,
 'eval_steps_per_second': 13.819,
 'epoch': 4.93}